# Deutsch-Jozsa algorithm

The **Deutsch–Jozsa algorithm** quantum kata is a series of exercises designed
to get you familiar with programming in Q#.

It covers the following topics:
* writing oracles (quantum operations which implement certain classical functions),
* Bernstein-Vazirani algorithm for recovering the parameters of a scalar product function,
* Deutsch-Jozsa algorithm for recognizing a function as constant or balanced, and
* writing tests in Q#.

You can read more about the quantum oracles, Deutsch and Deutsch-Jozsa algorithms in the [ExploringDeutschJozsaAlgorithm tutorial](../tutorials/ExploringDeutschJozsaAlgorithm/DeutschJozsaAlgorithmTutorial_P1.ipynb).

Each task is wrapped in one operation preceded by the description of the task.
Your goal is to fill in the blanks (marked with `// ...` comments)
with some Q# code that solves the task. To verify your answer, run the cell with Ctrl+Enter (⌘+Enter on macOS).

## Part I. Oracles

In this section you will implement oracles defined by classical functions using the following rules:
 - a function $f\left(x_0, ..., x_{N-1}\right)$ with N bits of input $x = \left(x_0, ..., x_{N-1}\right)$ and 1 bit of output $y$
   defines an oracle which acts on N input qubits and 1 output qubit.
 - the oracle effect on qubits in computational basis states is defined as follows:
   $|x\rangle |y\rangle \to |x\rangle |y \oplus f(x)\rangle$   ($\oplus$ is addition modulo 2).
 - the oracle effect on qubits in superposition is defined following the linearity of quantum operations.
 - the oracle must act properly on qubits in all possible input states.
 
You can read more about quantum oracles in [Q# documentation](https://docs.microsoft.com/azure/quantum/concepts-oracles).

### Task 1.1. $f(x) = 0$

**Inputs:** 
1. N qubits in an arbitrary state $|x\rangle$ (input register)
2. a qubit in an arbitrary state $|y\rangle$ (output qubit)


**Goal:**  transform state $|x, y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2).

In [1]:
%kata T11_Oracle_Zero 

operation Oracle_Zero (x : Qubit[], y : Qubit) : Unit {
    // Since f(x) = 0 for all values of x, |y ⊕ f(x)⟩ = |y⟩.
    // This means that the operation doesn't need to do any transformation to the inputs.
    
    // Run the cell (using Ctrl/⌘ + Enter) to see that the test passes.
}

Success!

### Task 1.2. $f(x) = 1$

**Inputs:** 
1. N qubits in an arbitrary state $|x\rangle$ (input register)
2. a qubit in an arbitrary state $|y\rangle$ (output qubit)


**Goal:**  transform state $|x, y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2).

<br/>
<details>
  <summary><b>Need a hint? Click here</b></summary>
  Since $f(x) = 1$ for all values of x, $|y \oplus f(x)\rangle = |y \oplus 1\rangle = |NOT y\rangle$.
  This means that the operation needs to flip qubit y (i.e. transform $|0\rangle$ to $|1\rangle$ and vice versa).
</details>

In [4]:
%kata T12_Oracle_One 

operation Oracle_One (x : Qubit[], y : Qubit) : Unit {
    // Since f(x) = 1 for all values of x, |y ⊕ f(x)⟩ = |y ⊕ 1⟩ = |NOT y⟩.
    // This means that the operation needs to flip qubit y (i.e. transform |0⟩ to |1⟩ and vice versa).
    // Addition modulo 2 is a binary operation for adding two numbers with the CNOT gate.
    // For f(x) = 0, CNOT does nothing. For f(x) = 1, CNOT flips the target.
    X(y);
}

Success!

### Task 1.3. $f(x) = x_k$ (the value of k-th qubit)

**Inputs:** 
1. N qubits in an arbitrary state $|x\rangle$ (input register)
2. a qubit in an arbitrary state $|y\rangle$ (output qubit)
3. 0-based index of the qubit from input register ($0 \le k < N$)

**Goal:**  transform state $|x, y\rangle$ into state $|x, y \oplus x_k\rangle$ ($\oplus$ is addition modulo 2).

In [5]:
%kata T13_Oracle_Kth_Qubit 

open Microsoft.Quantum.Diagnostics;

operation Oracle_Kth_Qubit (x : Qubit[], y : Qubit, k : Int) : Unit {
    // The following line enforces the constraints on the value of k that you are given.
    // You don't need to modify it. Feel free to remove it, this won't cause your code to fail.
    EqualityFactB(0 <= k and k < Length(x), true, "k should be between 0 and N-1, inclusive");
    // Addition modulo 2 is a binary operation for adding two numbers with the CNOT gate.
    // For f(x) = 0, CNOT does nothing. For f(x) = 1, CNOT flips the target.
    CNOT(x[k], y);
}

Success!

### Task 1.4. f(x) = 1 if x has odd number of 1s, and 0 otherwise

**Inputs:** 
1. N qubits in an arbitrary state $|x\rangle$ (input register)
2. a qubit in an arbitrary state $|y\rangle$ (output qubit)


**Goal:**  transform state $|x, y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2).

<br/>
<details>
  <summary><b>Need a hint? Click here</b></summary>
  $f(x)$ can be represented as $x_0 \oplus x_1 \oplus ... \oplus x_{N-1}$.
</details>

In [53]:
%kata T14_Oracle_OddNumberOfOnes

open Microsoft.Quantum.Canon;

operation Oracle_OddNumberOfOnes (x : Qubit[], y : Qubit) : Unit {
    // Addition modulo 2 is a binary operation for adding two numbers with the CNOT gate.
    // For f(x) = 0, CNOT does nothing. For f(x) = 1, CNOT flips the target.
    ApplyToEachCA(CNOT(_, y), x);
    //for q in x { CNOT(q, y); }
}

Success!

### Task 1.5. $f(x) = \bigoplus\limits_{i=0}^{N-1} r_i x_i$ for a given bit vector r (scalar product function)

**Inputs:** 
1. N qubits in an arbitrary state $|x\rangle$ (input register)
2. a qubit in an arbitrary state $|y\rangle$ (output qubit)
3. a bit vector of length N represented as an `Int[]`.
   You are guaranteed that the qubit array and the bit vector have the same length.

**Goal:**  transform state $|x, y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2).

In [9]:
%kata T15_Oracle_ProductFunction

open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Arrays;

operation Oracle_ProductFunction (x : Qubit[], y : Qubit, r : Int[]) : Unit {
    // The following line enforces the constraint on the input arrays.
    // You don't need to modify it. Feel free to remove it, this won't cause your code to fail.
    EqualityFactI(Length(x), Length(r), "Arrays should have the same length");
    // Addition modulo 2 is a binary operation for adding two numbers with the CNOT gate.
    // For f(x) = 0, CNOT does nothing. For f(x) = 1, CNOT flips the target.
    for i in IndexRange(r) {
        // We only get f(x) = 1 when bit == 1 and x == 1 state
        if (r[i] == 1) {
            CNOT(x[i], y);
        }
    }
}

Success!

### Task 1.6. $f(x) = \bigoplus\limits_{i=0}^{N-1} \left(r_i x_i + (1 - r_i) (1 - x_i) \right)$ for a given bit vector r (scalar product function)

**Inputs:** 
1. N qubits in an arbitrary state $|x\rangle$ (input register)
2. a qubit in an arbitrary state $|y\rangle$ (output qubit)
3. a bit vector of length N represented as an `Int[]`.
   You are guaranteed that the qubit array and the bit vector have the same length.

**Goal:**  transform state $|x, y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2).

<br/>
<details>
  <summary><b>Need a hint? Click here</b></summary>
  Since each addition is done modulo 2, you can evaluate the effect of each term independently$.
</details>

In [11]:
%kata T16_Oracle_ProductWithNegationFunction

open Microsoft.Quantum.Diagnostics;

operation Oracle_ProductWithNegationFunction (x : Qubit[], y : Qubit, r : Int[]) : Unit {
    // The following line enforces the constraint on the input arrays.
    // You don't need to modify it. Feel free to remove it, this won't cause your code to fail.
    EqualityFactI(Length(x), Length(r), "Arrays should have the same length");
    // Addition modulo 2 is a binary operation for adding two numbers with the CNOT gate.
    // For f(x) = 0, CNOT does nothing. For f(x) = 1, CNOT flips the target.
    for i in IndexRange(r) {
        if (r[i] == 1) { 
            CNOT(x[i], y); 
        } 
        else { 
            ControlledOnInt(0, X)(x[i..i], y); 
        }
    }
}

Success!

### Task 1.7. $f(x) = \bigoplus\limits_{i=0}^{N-1} x_i + $ (1 if prefix of x is equal to the given bit vector, and 0 otherwise) modulo 2

**Inputs:** 
1. N qubits in an arbitrary state $|x\rangle$ (input register)
2. a qubit in an arbitrary state $|y\rangle$ (output qubit)
3. a bit vector of length $K$ represented as an `Int[]` ($1 \le K \le N$).

**Goal:**  transform state $|x, y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2).

> A prefix of length K of a state $|x\rangle = |x_0, ..., x_{N-1}\rangle$ is the state of its first K qubits $|x_0, ..., x_{K-1}\rangle$. For example, a prefix of length 2 of a state $|0110\rangle$ is 01.

<details>
  <summary><b>Need a hint? Click here</b></summary>
  The first term is the same as in task 1.4. To implement the second term, you can use `Controlled` functor which allows to perform multicontrolled gates (gates with multiple control qubits).
</details>

In [35]:
%kata T17_Oracle_HammingWithPrefix

open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Logical;

operation Oracle_HammingWithPrefix (x : Qubit[], y : Qubit, prefix : Int[]) : Unit {
    // The following line enforces the constraint on the input arrays.
    // You don't need to modify it. Feel free to remove it, this won't cause your code to fail.
    let K = Length(prefix);
    EqualityFactB(1 <= K and K <= Length(x), true, "K should be between 1 and N, inclusive");
    // Addition modulo 2 is a binary operation for adding two numbers with the CNOT gate.
    // For f(x) = 0, CNOT does nothing. For f(x) = 1, CNOT flips the target.
    for i in IndexRange(x) {
        CNOT(x[i], y);
        if (i == K - 1) {
            ControlledOnBitString(Mapped(EqualI(_, 1), prefix), X)(x[...i], y);
        }
    }
}

Success!

### Task 1.8. f(x) = 1 if x has two or three bits (out of three) set to 1, and 0 otherwise (majority function)

**Inputs:** 
1. 3 qubits in an arbitrary state $|x\rangle$ (input register)
2. a qubit in an arbitrary state $|y\rangle$ (output qubit)


**Goal:**  transform state $|x, y\rangle$ into state $|x, y \oplus f(x)\rangle$ ($\oplus$ is addition modulo 2).

<br/>
<details>
  <summary><b>Need a hint? Click here</b></summary>
  Represent $f(x)$ in terms of AND and $\oplus$ operations.
</details>

In [37]:
%kata T18_Oracle_MajorityFunction

open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Logical;

operation Oracle_MajorityFunction (x : Qubit[], y : Qubit) : Unit {
    // The following line enforces the constraint on the input array.
    // You don't need to modify it. Feel free to remove it, this won't cause your code to fail.
    EqualityFactI(3, Length(x), "x should have exactly 3 qubits");
    // Not comfortable with the exponential time complexity in the general case of N-qubit input register...
    //for i in 0 .. PowI(2, Length(x)) - 1 {
    //    if (Count(EqualB(_, true), IntAsBoolArray(i, Length(x)))) > 1) {
    //        ControlledOnInt(i, X)(x, y);
    //    }
    //}
    // General case? i.e. N = 9
    CCNOT(x[0], x[1], y);
    CCNOT(x[0], x[2], y);
    CCNOT(x[1], x[2], y);
}

Success!

## Part II. Deutsch-Jozsa Algorithm

In this section you will implement the Deutsch-Jozsa algorithm and run it on the oracles you've defined in part I to observe the results. 

This algorithm solves the following problem. You are given a quantum oracle which implements a classical function $f(x): \{0, 1\}^N \to \{0, 1\}$. You are guaranteed that the function $f$ is either constant (has the same value for all inputs) or balanced (has value 0 for half of the inputs and 1 for the other half of the inputs). The goal of the algorithm is to figure out whether the function is constant or balanced in just one oracle call.
 
* You can read more about the Deutsch-Jozsa algorithms and explore its finer points in the [ExploringDeutschJozsaAlgorithm tutorial](../tutorials/ExploringDeutschJozsaAlgorithm/DeutschJozsaAlgorithmTutorial.ipynb).
* You can read more about the Deutsch-Jozsa algorithm in [Wikipedia](https://en.wikipedia.org/wiki/Deutsch%E2%80%93Jozsa_algorithm).
* [Lecture 5: A simple searching algorithm; the Deutsch-Jozsa algorithm](https://cs.uwaterloo.ca/~watrous/QC-notes/QC-notes.05.pdf).

### Task 2.1. State preparation for Deutsch-Jozsa algorithm

**Inputs:**
1. N qubits in $|0\rangle$ state (query register)
2. a qubit in $|0\rangle$ state (answer register)

**Goal:**
1. Prepare an equal superposition of all basis vectors from $|0...0\rangle$ to $|1...1\rangle$ on query register
   (i.e., state $\frac{1}{\sqrt{2^N}}\big(|0...0\rangle + ... + |1...1\rangle\big)$ ).
2. Prepare $|-\rangle = \frac{1}{\sqrt2} (|0\rangle - |1\rangle)$ state on answer register.    

In [38]:
%kata T21_DJ_StatePrep

operation DJ_StatePrep (query : Qubit[], answer : Qubit) : Unit is Adj {
    //
    ApplyToEachCA(H, query);
    X(answer);
    H(answer);
}

Success!

### Task 2.2. Deutsch-Jozsa Algorithm

**Inputs:** 
1. the number of qubits $N$ in the input register for the function f
2. a quantum operation which implements the oracle $|x, y\rangle \to |x, y \oplus f(x)\rangle$, where x is an $N$-qubit input register, y is a 1-qubit answer register, and f is a Boolean function


**Output:**  `true` if the function f is constant, or `false` if the function f is balanced.

In [41]:
%kata T22_DJ_Algorithm

open Microsoft.Quantum.Measurement;

operation DJ_Algorithm (N : Int, oracle : ((Qubit[], Qubit) => Unit)) : Bool {
    // Allocate an array of N qubits for the input register x and one qubit for the answer register y.
    use (x, y) = (Qubit[N], Qubit());

    within { DJ_StatePrep(x, y); }
    apply { oracle(x, y); }

    // A constant function simply applies a global phase to the wave function, which is unphysical.
    let measurementOutcome = ResultArrayAsInt(MultiM(x)) == 0;
    Reset(y);
    return measurementOutcome;
}

Success!

### Task 2.3. Running Deutsch-Jozsa Algorithm

**Goal**: Use your implementation of Deutsch-Jozsa algorithm from task 2.1 to test each of the oracles you've implemented in part I for being constant or balanced.

> This is an open-ended task, and is not covered by a unit test. To run the code, execute the cell with the definition of the `Run_DeutschJozsa_Algorithm` operation first; if it compiled successfully without any errors, you can run the operation by executing the next cell (`%simulate Run_DeutschJozsa_Algorithm`).

> Note that this task relies on your implementations of the previous tasks. If you are getting the "No variable with that name exists." error, you might have to execute previous code cells before retrying this task.

In [60]:
open Microsoft.Quantum.Diagnostics;

operation Run_DeutschJozsa_Algorithm () : String {
    // You can use the Fact function to check that the return value of DJ_Algorithm operation matches the expected value.
    // Uncomment the next line to test the algorithm on the oracle for f(x) = 0.
    Fact(DJ_Algorithm(4, Oracle_Zero), "f(x) = 0 not identified as constant");
    Fact(DJ_Algorithm(4, Oracle_One), "f(x) = 1 not identified as constant");
    Contradiction(DJ_Algorithm(4, Oracle_OddNumberOfOnes), "f(x) not identified as balanced");
    Contradiction(DJ_Algorithm(3, Oracle_MajorityFunction), "f(x) not identified as balanced");
    
    // If all tests pass, report success!
    return "Success!";
}

Run_DeutschJozsa_Algorithm

In [61]:
%simulate Run_DeutschJozsa_Algorithm

Success!

## Part III. Bernstein–Vazirani Algorithm

In this section you will implement the Bernstein-Vazirani algorithm and run it on the oracles you've defined in part I to observe the results. 

This algorithm solves the following problem. You are given a quantum oracle which implements a classical function $f(x): \{0, 1\}^N \to \{0, 1\}$. You are guaranteed that the function $f$ can be represented as a scalar product, i.e., there exists a bit vector $r = (r_0, ..., r_{N-1})$ such that $f(x) = \bigoplus \limits_{i=0}^{N-1} x_i r_i$. The goal of the algorithm is to reconstruct the bit vector $r$ in just one oracle call.
 
You can read more about the Bernstein-Vazirani algorithm in ["Quantum Algorithm Implementations for Beginners"](https://arxiv.org/abs/1804.03719), section III.

### Task 3.1. Bernstein-Vazirani Algorithm

**Inputs:** 
1. the number of qubits $N$ in the input register for the function f
2. a quantum operation which implements the oracle $|x, y\rangle \to |x, y \oplus f(x)\rangle$, where x is an $N$-qubit input register, y is a 1-qubit answer register, and f is a Boolean function


**Output:**  The bit vector $r$ reconstructed from the oracle.

In [84]:
%kata T31_BV_Algorithm

operation BV_Algorithm (N : Int, oracle : ((Qubit[], Qubit) => Unit)) : Int[] {
    // The algorithm is very similar to Deutsch-Jozsa algorithm; try to implement it without hints.
    use (x, y) = (Qubit[N], Qubit());

    within { DJ_StatePrep(x, y); }
    apply { oracle(x, y); }
    
    mutable r = [0, size = N];
    for i in 0 .. N - 1 { set r w/= i <- MResetZ(x[i]) == Zero ? 0 | 1; }
    return r;
}

Success!

### Task 3.2. Running Bernstein-Vazirani Algorithm

**Goal**: Use your implementation of Bernstein-Vazirani algorithm from task 3.1 to reconstruct the hidden vector $r$ for the oracles you've implemented in part I.

> This is an open-ended task, and is not covered by a unit test. To run the code, execute the cell with the definition of the `Run_BernsteinVazirani_Algorithm` operation first; if it compiled successfully without any errors, you can run the operation by executing the next cell (`%simulate Run_BernsteinVazirani_Algorithm`).

> Note that this task relies on your implementations of the previous tasks. If you are getting the "No variable with that name exists." error, you might have to execute previous code cells before retrying this task.

<details>
  <summary><b>Need a hint? Click here</b></summary>
  Not all oracles from part I can be represented as scalar product functions. The most generic oracle you can use in this task is Oracle_ProductFunction from task 1.5; Oracle_Zero, Oracle_Kth_Qubit and Oracle_OddNumberOfOnes are special cases of this oracle.
</details>

In [97]:

open Microsoft.Quantum.Diagnostics;

operation Run_BernsteinVazirani_Algorithm () : String {
    // Now use the library function AllEqualityFactI in Microsoft.Quantum.Diagnostics to verify the results of the algorithm.
    // Uncomment the following lines to test the algorithm on the oracle for f(x) = 0.
    AllEqualityFactI(BV_Algorithm(3, Oracle_Zero), [0, 0, 0], "Incorrect result for f(x) = 0");
    AllEqualityFactI(BV_Algorithm(3, Oracle_One), [0, 0, 0], "Incorrect result for f(x) = 1");
    let r = [0, 1, 0];
    let oracle = Oracle_ProductFunction(_, _, r);
    AllEqualityFactI(BV_Algorithm(3, oracle), r, "Incorrect result for f(x)");

    //let oracleTwo = Oracle_ProductWithNegationFunction(_, _, r);
    //AllEqualityFactI(BV_Algorithm(3, oracleTwo), r, "Incorrect result for f(x)");
    
    // If all tests pass, report success!
    return "Success!";
}

Run_BernsteinVazirani_Algorithm

In [98]:
%simulate Run_BernsteinVazirani_Algorithm

Success!

## Part IV. Come up with your own algorithm!

In this section you will come up with your own algorithm to solve a problem similar to the one described in part III. 

The problem is formulated as follows. You are given a quantum oracle which implements a classical function $f(x): \{0, 1\}^N \to \{0, 1\}$. You are guaranteed that there exists a bit vector $r = (r_0, ..., r_{N-1})$ such that the function $f$ can be represented as follows: $f(x) = \bigoplus \limits_{i=0}^{N-1} \left( x_i r_i + (1 - x_i)(1 - r_i) \right)$. You have to reconstruct the bit vector $r$ in just one oracle call.

> Note that you have implemented the oracle for this function in task 1.6.

### Task 4. Noname Algorithm

**Inputs:** 
1. the number of qubits $N$ in the input register for the function f
2. a quantum operation which implements the oracle $|x, y\rangle \to |x, y \oplus f(x)\rangle$, where x is an $N$-qubit input register, y is a 1-qubit answer register, and f is a Boolean function

**Output:**  Any bit vector $r$ that would generate the same oracle as the one you are given.

<br/>
<details>
  <summary><b>Need a hint? Click here</b></summary>
  For each oracle there are multiple bit vectors that generate it; it is sufficient to find any one of them.
</details>

In [ ]:
%kata T41_Noname_Algorithm

operation Noname_Algorithm (N : Int, oracle : ((Qubit[], Qubit) => Unit)) : Int[] {
    // The algorithm is very similar to Bernstein-Vazirani algorithm; try to implement it without hints.
    // ...
}